# Simulateur de traffic routier

**Objectif** : Simuler le traffic dans le cas suivant

<img src="Banque d'image interface\Cas_etude_feux.png" width="500"/>


## Partie 1 : Création de la matrice qui modélisera notre route

### Syntaxte

Il faut voir le schéma du haut comme une compilation de blocs de différentes catégories routes, feux rouges, intersection, cedez le passage, debut, fin etc ...

Ainsi, notre matrice retraduira cet assemblage de blocs afin d'éviter la création de trop grosses matrices (si on redessine toute la route). Chaque élement de la matrice sera donc un élement de la route et devra respecter la syntaxe suivante :

Element = ["Type" = str, Caractéristique du type = int]

Détail pour chaque élements :

- route = ['Route', proba [0,1,0,0], sens de circulation 0 = ouest, 1 = sud ,2 = est, 3 = nord]
- intersection = ['Intersection']
- Feux rouges = ['Feu',proba [0,1,0,0], sens de circulation 0 = ouest, 1 = sud ,2 = est, 3 = nord, cycles en secondes, état des feux rouges]
- priorité = ['Priorite, proba [0,1,0,0], sens de circulation 0 = ouest, 1 = sud ,2 = est, 3 = nord]
- Début = ['Debut', proba [0,1,0,0], sens de circulation 0 = ouest, 1 = sud ,2 = est, 3 = nord, débit = int]
- Fin = ['Fin']

Cependant ces élements ne permetent pas de représenter ni de traiter l'évolution du traffic. C'est pourquoi il faut joindre à ces blocs des blocs 'transport' permettant de modéliser l'évolution du traffic en temps réel.

C'est blocs transport se modéliseront de le manière suivantes :

Transport = [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1]

Les 0 sont des espaces vides et les 1 des emplacements occupés. La capacité de l'élement précédant le bloc transport définira la longeur du bloc transport.

Enfin, à chaque itération la suite de 0 et de 1 se déplacera vers la droite et lorsqu'un 1 atteindra le bord droit on ajoutera 1 à l'ocuppation du bloc suivant.

On ajoutera c'est bloc transport à la fin des élements

On sépare tout en 3

### Hypothèses

Les voitures ne se déplace pas dinstantanément il y a un effet de 'chaine', ainsi il faudra à minima un 0 devant un & pour qu'il puisse avancer.

### Retour à notre cas

En apliquant le découpage on a :

<img src="Banque d'image interface\Decoupage_route.jpg" width="500"/>


In [1]:
# Cas 1 : feux rouges

Route_01 = ["Fin"]
Route_02 = ["Intersection"]
Route_03 = ["Intersection"]
Route_04 = ["Route", 0]
Route_05 = ["Depart", 0, 5]
Route_10 = ["Depart", 2, 5]
Route_11 = ["Route", 2]
Route_12 = ["Intersection"]
Route_13 = ["Intersection"]
Route_14 = ["Fin"]
Route_22 = ["Fin"]
Route_23 = ["Feu", 3, 20, False]
Route_33 = ["Route", 3]
Route_43 = ["Depart", 3, 5]

Traffic_01 = [0]
Traffic_02 = [0]
Traffic_03 = [0]
Traffic_04 = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Traffic_05 = [0]
Traffic_10 = [0]
Traffic_11 = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Traffic_12 = [0]
Traffic_13 = [0]
Traffic_14 = [0]
Traffic_22 = [0]
Traffic_23 = [0]
Traffic_33 = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Traffic_43 = [0]

Direction_01 = []
Direction_02 = []
Direction_03 = []
Direction_04 = [[1, 0, 0, 0], 0]
Direction_05 = [[1, 0, 0, 0], 0]
Direction_10 = [[0, 0, 1, 0], 0]
Direction_11 = [[0, 1, 0, 0], 0]
Direction_12 = []
Direction_13 = []
Direction_14 = []
Direction_22 = []
Direction_23 = [[0, 0, 1, 0], 0]
Direction_33 = [[0, 0, 0, 1], 3]
Direction_43 = [[0, 0, 0, 1], 0]

Route_etude = [
    [0,0,0,0,0,0],
    [0, Route_01, Route_02, Route_03, Route_04, Route_05],
    [Route_10, Route_11, Route_12, Route_13, Route_14, 0],
    [0, 0, Route_22, Route_23, 0, 0],
    [0, 0, 0, Route_33, 0, 0],
    [0, 0, 0, Route_43, 0, 0],
]

Traffic_etude = [
    [0,0,0,0,0,0],
    [0, Traffic_01, Traffic_02, Traffic_03, Traffic_04, Traffic_05],
    [Traffic_10, Traffic_11, Traffic_12, Traffic_13, Traffic_14, 0],
    [0, 0, Traffic_22, Traffic_23, 0, 0],
    [0, 0, 0, Traffic_33, 0, 0],
    [0, 0, 0, Traffic_43, 0, 0],
]

Direction_etude = [
    [0,0,0,0,0,0],
    [0, Direction_01, Direction_02, Direction_03, Direction_04, Direction_05],
    [Direction_10, Direction_11, Direction_12, Direction_13, Direction_14, 0],
    [0, 0, Direction_22, Direction_23, 0, 0],
    [0, 0, 0, Direction_33, 0, 0],
    [0, 0, 0, Direction_43, 0, 0],
]

temps = 0

## Modèle physique

Retouver graph dans la doc mais en gros un utlisateur avance si devatn lui il y a au moins une distance suffisante + 1s de temps de réponse.

On a besoin de créer un module qui permet de faire bouger des voitures selon ces régles dans le bloc transport de chaque éléments.

Pour rappel le mouvement des voitures se fera de la gauche ver la droite ainsi la première voiture à bouger est la dernière dans la liste.

Chaque itération vaut 1 seconde.


In [2]:
# Tous les imports et variables

import copy
import time
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import random

### Faire vivre la route

In [3]:
def update_direction(route, direction):

    for i in range(len(route)):
        for j in range(len(route[i])):
            if route[i][j] != 0:
                if route[i][j][0] != "Fin" and route[i][j][0] != "Intersection":
                    direction[i][j][1] = random.choices(
                        [0, 1, 2, 3], weights=direction[i][j][0], k=1
                    )[0]


def update_départ(route, traffic,temps):

    for i in range(len(route)):
        for j in range(len(route[i])):
            if route[i][j] != 0:
                if route[i][j][0] == "Depart":
                    if (temps % route[i][j][2]) == 0:
                        traffic[i][j][0] += 1


def update_feux_rouges(route,temps):

    for i in range(len(route)):
        for j in range(len(route[i])):
            if route[i][j] != 0:
                if route[i][j][0] == "Feu":
                    if (temps % route[i][j][2]) == 0:
                        route[i][j][3] = not (route[i][j][3])

def update_route(route,direction,traffic,temps):
    update_direction(route, direction)
    update_départ(route, traffic,temps)
    update_feux_rouges(route,temps)

"""Fonction qui retourne le chemin pour sortir des intersections"""

def chemin_intersection(route, direction, pos_x, pos_y):

    pos = route[pos_x][pos_y][1]
    dir = direction[pos_x][pos_y][1]

    if pos == dir:
        return [pos, pos]

    if pos == (dir + 1) % 4:
        return [dir]

    if dir == (pos + 1) % 4:
        return [pos, dir, dir]

Premiere étape faire passer les voitures qui sont arrivées à la fin de l'élement vers les blocs adjacents libres  
On vient examiner chaque élément de la route  
Chaque élémént à sa particularité:  
Route il suffit que l'espace adjacent soit libre  
Feu rouge il faut que le feu soit vert et qu'il n'y est personne en face  
Priorité en fonction des intention de l'utilisateur il faut que il n'y est personne sur l'extrémitée des routes qui coupe la priorité  
Intersection en fonction des intentions de l'utilisateur il faut que le bloc concerné soit libre  
Départ et fin à part  
La fin est un bloc tout le temps disponible  
Plus règle de la prioritée à droite  
Il faudrait faire l'inventaire des voitures qui veulent changer d'élément avec destination associé  
Les éléments devrait également être classé par ordre de prorité  
Syntaxte des éléments des priorité [pos de départ, pos d'arrivé, vers intersection ]
Le classement des prioirités ne concerne que les blocs qui veulent aller sur les blocs intersections
Chaque utilisateur est indépendant

Cette fonction prioruté est la fonction la plus suceptible d'améliroation afin de prendre en compte au maximum de l'aspect humain

'''Fonction pour les priorités objectif on veut un tableau qui classe par ordre croissant de priorité les inteentiosn des utilisateurs
voulant changer de bloc'''
'''Etape 1 : Faire l'inventaire de toutes les intentiosn des utilisateurs'''
'''Attention on dit que l'utilsateur à une intention si il peut réaliser sont action c'est à dire si les conditons définit précedement'''
'''Sont remplis'''

fonction chemin_intersection à développer plus tard pour extrapoler a plus de 4 blocs intersections collé


In [4]:
"""Donne la liste des intentions"""


def intentions(route, direction, traffic):
    res = []
    for i in range(len(route)):
        for j in range(len(route[i])):
            if route[i][j] != 0:
                if traffic[i][j][-1] >= 1:
                    """Pour la route et les départ la voiture voudra toujour suivre la direction aucune perturbation n'est à prévoir"""
                    """Les questions de priorité se pose généralement sur les bloc intersections ou feu rouge ou priorité"""
                    if route[i][j][0] == "Route" or route[i][j][0] == "Depart":
                        dir_route = route[i][j][1]
                        if dir_route == 0:
                            res.append([i, j, i, j - 1])
                        elif dir_route == 1:
                            res.append([i, j, i + 1, j])
                        elif dir_route == 2:
                            res.append([i, j, i, j + 1])
                        else:
                            res.append([i, j, i - 1, j])

                    """Pour les intersections ca se complique en effet il faut faire attention au priorité à droite"""
                    """La prioirité à droite s'applique si le bloc à droite de la destination est une route occupée"""
                    """De ce fait sur une intersection on peut avancer si le bloc à droite de la destination n'est pas une route"""
                    """Ou alors c'est une route non occupée"""
                    """Dépend de la direction et c'est la où c'est plus chaud matrice des chemins"""
                    if route[i][j][0] == "Intersection":
                        if traffic[i][j][0] == 1:
                            dir_inter = direction[i][j][0]
                            if dir_inter == 0:
                                if 0 <= i - 1 <= len(route) and 0 <= j - 1 <= len(
                                    route[i]
                                ):
                                    if route[i - 1][j - 1] != 0:
                                        if route[i - 1][j - 1][0] != "Route":
                                            res.append([i, j, i, j - 1])
                                        elif traffic[i - 1][j - 1][-1] == 0:
                                            res.append([i, j, i, j - 1])
                                    else:
                                        res.append([i, j, i, j - 1])
                                else:
                                    res.append([i, j, i, j - 1])
                            elif dir_inter == 1:
                                if 0 <= i + 1 <= len(route) and 0 <= j - 1 <= len(
                                    route[i]
                                ):
                                    if route[i + 1][j - 1] != 0:
                                        if route[i + 1][j - 1][0] != "Route":
                                            res.append([i, j, i + 1, j])
                                        elif traffic[i + 1][j - 1][-1] == 0:
                                            res.append([i, j, i + 1, j])
                                    else:
                                        res.append([i, j, i + 1, j])
                                else:
                                    res.append([i, j, i + 1, j])
                            elif dir_inter == 2:
                                if 0 <= i + 1 <= len(route) and 0 <= j + 1 <= len(
                                    route[i]
                                ):
                                    if route[i + 1][j + 1] != 0:
                                        if (
                                            route[i + 1][j + 1][0] != "Route"
                                            or traffic[i + 1][j + 1][-1] == 0
                                        ):
                                            res.append([i, j, i, j + 1])
                                    else:
                                        res.append([i, j, i, j + 1])
                                else:
                                    res.append([i, j, i, j + 1])
                            else:
                                if 0 <= i - 1 <= len(route) and 0 <= j + 1 <= len(
                                    route[i]
                                ):
                                    if route[i - 1][j + 1] != 0:
                                        if (
                                            route[i - 1][j + 1][0] != "Route"
                                            or traffic[i - 1][j + 1][-1] == 0
                                        ):
                                            res.append([i, j, i - 1, j])
                                    else:
                                        res.append([i, j, i - 1, j])
                                else:
                                    res.append([i, j, i - 1, j])

                    """Feux rouges : pour les feux rouges il va dans la direction si le feu est vert"""
                    """Attention execption si le feu rouge est relier à une intersection"""
                    """Et que celle-ci est pleine (voiture sur la gauche) alors on le laisse passer = évite les blocages"""
                    if route[i][j][0] == "Feu":
                        if route[i][j][-1] == True:
                            dir_feu = route[i][j][1]
                            if dir_feu == 0:
                                if 0 <= i + 1 <= len(route) and 0 <= j - 1 <= len(
                                    route[i]
                                ):
                                    if route[i][j - 1] != 0:
                                        if (
                                            route[i][j - 1][0] == "Intersection"
                                            and traffic[i + 1][j - 1][-1] == 0
                                        ):
                                            res.append([i, j, i, j - 1])
                                    else:
                                        res.append([i, j, i, j - 1])
                                else:
                                    res.append([i, j, i, j - 1])
                            elif dir_feu == 1:
                                if 0 <= i + 1 <= len(route) and 0 <= j + 1 <= len(
                                    route[i]
                                ):
                                    if route[i + 1][j] != 0:
                                        if (
                                            route[i + 1][j][0] == "Intersection"
                                            and traffic[i + 1][j + 1][-1] == 0
                                        ):
                                            res.append([i, j, i + 1, j])
                                    else:
                                        res.append([i, j, i + 1, j])
                                else:
                                    res.append([i, j, i + 1, j])
                            elif dir_feu == 2:
                                if 0 <= i - 1 <= len(route) and 0 <= j + 1 <= len(
                                    route[i]
                                ):
                                    if route[i][j + 1] != 0:
                                        if (
                                            route[i][j + 1][0] == "Intersection"
                                            and traffic[i - 1][j + 1][-1] == 0
                                        ):
                                            res.append([i, j, i, j + 1])
                                    else:
                                        res.append([i, j, i, j + 1])
                                else:
                                    res.append([i, j, i, j + 1])
                            else:
                                if 0 <= i - 1 <= len(route) and 0 <= j - 1 <= len(
                                    route[i]
                                ):
                                    if route[i - 1][j] != 0:
                                        if (
                                            route[i - 1][j][0] == "Intersection"
                                            and traffic[i - 1][j - 1][-1] == 0
                                        ):
                                            res.append([i, j, i - 1, j])
                                    else:
                                        res.append([i, j, i - 1, j])
                                else:
                                     res.append([i, j, i - 1, j])

                    """Pour les priorités suivant la direction il faut :"""
                    """Si le choix de la desico, implique de couper 1 seule route alors il faut qu'il n'y est personne sur la voie coupé"""
                    """On estime qu'une voie est libre si il y a 3 emplacement vacants"""
                    """Si le choix implique de couper """
                    if route[i][j][0] == "Priorite":
                        sens_prio = route[i][j][1]
                        dir_prio = direction[i][j][1]
                        """Cas 1 voie coupé """
                        if dir_prio == (sens_prio + 1) % 4:
                            if sens_prio == 0:
                                if (
                                    0 <= i + 1 <= len(route)
                                    and 0 <= i + 2 <= len(route)
                                    and 0 <= j - 1 <= len(route[i])
                                ):
                                    if (
                                        traffic[i + 1][j - 1][-1] == 0
                                        and traffic[i + 2][j - 1][-1] == 0
                                        and traffic[i + 2][j - 1][-2] == 0
                                    ):  # ATTENTION NE MARCHE QUE POUR NOTRE CAS A MODIFIER SI GENERALISATION
                                        res.append([i, j, i, j - 1])
                            elif sens_prio == 1:
                                if (
                                    0 <= i + 1 <= len(route)
                                    and 0 <= j + 1 <= len(route)
                                    and 0 <= j + 2 <= len(route[i])
                                ):
                                    if (
                                        traffic[i + 1][j + 1][-1] == 0
                                        and traffic[i + 1][j + 2][-1] == 0
                                        and traffic[i + 1][j + 2][-2] == 0
                                    ):  # ATTENTION NE MARCHE QUE POUR NOTRE CAS A MODIFIER SI GENERALISATION
                                        res.append([i, j, i + 1, j])
                            elif sens_prio == 2:
                                if (
                                    0 <= i - 1 <= len(route)
                                    and 0 <= i - 2 <= len(route)
                                    and 0 <= j + 1 <= len(route[i])
                                ):
                                    if (
                                        traffic[i - 1][j + 1][-1] == 0
                                        and traffic[i - 2][j + 1][-1] == 0
                                        and traffic[i - 2][j + 1][-2] == 0
                                    ):  # ATTENTION NE MARCHE QUE POUR NOTRE CAS A MODIFIER SI GENERALISATION
                                        res.append([i, j, i, j + 1])
                            else:
                                if (
                                    0 <= i - 1 <= len(route)
                                    and 0 <= j - 1 <= len(route)
                                    and 0 <= j - 2 <= len(route[i])
                                ):
                                    if (
                                        traffic[i - 1][j - 1][-1] == 0
                                        and traffic[i - 1][j - 2][-1] == 0
                                        and traffic[i - 1][j - 2][-2] == 0
                                    ):  # ATTENTION NE MARCHE QUE POUR NOTRE CAS A MODIFIER SI GENERALISATION
                                        res.append([i, j, i - 1, j])

    return res


### Mouvement

In [5]:
def mouvement(route, direction, traffic, temps):
    """Pour rappel la route est une matrice 2x2"""

    update_route(route, direction, traffic, temps)

    ref_traffic = copy.deepcopy(traffic)

    intention = intentions(route, direction, traffic)

    for i in range(len(intention)):
        x = intention[i][0]
        y = intention[i][1]
        nx = intention[i][2]
        ny = intention[i][3]

        # Cas particulier départ autres blocs
        if route[x][y][0] == "Depart":
            if traffic[nx][ny][0] == 0:
                traffic[nx][ny][0] += 1
                traffic[x][y][-1] -= 1

        # Cas particulier intersection->intersection
        elif route[nx][ny][0] == "Intersection" and route[x][y][0] == "Intersection":
            if traffic[nx][ny][0] == 0 and ref_traffic[nx][ny][0] == 0:
                direction[nx][ny] = direction[x][y][1::]
                direction[x][y] = []
                traffic[nx][ny][0] += 1
                traffic[x][y][-1] -= 1

            # Cas particulier non/intersection->intersection
        elif (
            route[nx][ny][0] == "Intersection" and route[x][y][0] != "Intersection"
        ):
            if traffic[nx][ny][0] == 0 and ref_traffic[nx][ny][0] == 0:
                print('a', chemin_intersection(route, direction, x, y))
                direction[nx][ny] = chemin_intersection(route, direction, x, y)
                traffic[nx][ny][0] += 1
                traffic[x][y][-1] -= 1

        # Cas particulier intersection->non-intersection
        elif route[nx][ny][0] != "Intersection" and route[x][y][0] == "Intersection":
            if route[nx][ny][0] == 'Fin':
                direction[x][y] = []
                traffic[nx][ny][0] += 1
                traffic[x][y][-1] -= 1
            elif traffic[nx][ny][0] == 0 and ref_traffic[nx][ny][0] == 0:    
                direction[x][y] = []
                traffic[nx][ny][0] += 1
                traffic[x][y][-1] -= 1

            # Cas général
        else:
            if route[nx][ny][0] == 'Fin':
                traffic[nx][ny][0] += 1
                traffic[x][y][-1] -= 1
            elif traffic[nx][ny][0] == 0 and ref_traffic[nx][ny][0] == 0:    
                traffic[nx][ny][0] += 1
                traffic[x][y][-1] -= 1

    for i in range(len(route)):
        for j in range(len(route)):
            if route[i][j] != 0:
                if route[i][j][0] == "Route":
                    for k in range(2, len(traffic[i][j]) + 1):
                        if (
                            traffic[i][j][-k] == 1
                            and traffic[i][j][-k + 1] == 0
                            and ref_traffic[i][j][-k + 1] == 0
                        ):
                            traffic[i][j][-k + 1] += 1
                            traffic[i][j][-k] -= 1

    temps += 1


def n_mouvement(route, direction, traffic, temps, n):

    for k in range(n):
        mouvement(route, direction, traffic, temps)
        print(k)
        print(traffic)
        

### Simulation

In [ ]:
def simulation(route,direction,traffic,temps,n):
    for i in range(n):
        mouvement(route, direction, traffic, temps)
        traffic_matrix = traffic
        
        # Taille de la matrice
        n_rows = len(traffic_matrix)
        n_cols = len(traffic_matrix[0])

        # Créer une figure et des axes
        fig, ax = plt.subplots(figsize=(10, 10))

        # Calcul du facteur de mise à l'échelle pour la largeur
        # Trouver la plus grande longueur de route dans la matrice
        max_route_length = max([len(item) if isinstance(item, list) and len(item) > 1 else 1 for row in traffic_matrix for item in row])

        # Parcourir la matrice et ajouter des éléments graphiques
        for i in range(n_rows):
            for j in range(n_cols):
                value = traffic_matrix[i][j]
                
                # Si c'est une case vide (0)
                if value == 0:
                    ax.add_patch(patches.Rectangle((j, n_rows - 1 - i), 1, 1, fill=True, color='white', edgecolor='black'))
                
                # Si c'est une voiture simple (1)
                elif value == [1]:
                    ax.add_patch(patches.Rectangle((j, n_rows - 1 - i), 1, 1, fill=True, color='black', edgecolor='black'))
                
                # Si c'est une route (liste avec plusieurs valeurs)
                elif isinstance(value, list) and len(value) > 1:
                    route_len = len(value)
                    route_color = 'Grey'  # Assigner une couleur différente à chaque route
                    # Adapter la largeur des sous-cases pour que toutes les sous-cases tiennent dans une case
                    sub_w = 1 / route_len  # Largeur de la sous-case
                    
                    # Diviser la case en plusieurs sous-cases pour chaque élément de la route
                    for k, sub_value in enumerate(value):
                        sub_x = j + (k * sub_w)  # Position en x de la sous-case
                        if sub_value == 1:
                            # Colorier la sous-case et son contour
                            ax.add_patch(patches.Rectangle((sub_x, n_rows - 1 - i), sub_w, 1, 
                                                        fill=True, color=route_color, edgecolor='Black'))
                        else:
                            # Colorier une sous-case vide avec un contour coloré
                            ax.add_patch(patches.Rectangle((sub_x, n_rows - 1 - i), sub_w, 1, 
                                                        fill=True, color='white', edgecolor='Black'))
                
                # Si c'est une zone de départ ou fin (nombre > 1)
                elif isinstance(value, list) and value[0] > 1:
                    ax.add_patch(patches.Rectangle((j, n_rows - 1 - i), 1, 1, fill=True, color='lightblue', edgecolor='black'))
                    # Annoter la case avec le nombre correspondant
                    ax.text(j + 0.5, n_rows - 1 - i + 0.5, str(value[0]), color='red', weight='bold', ha='center', va='center')

        # Réglages de la grille
        ax.set_xlim(0, n_cols)
        ax.set_ylim(0, n_rows)
        ax.set_xticks(np.arange(0, n_cols, 1))
        ax.set_yticks(np.arange(0, n_rows, 1))
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.grid(True)

        # Afficher la figure
        plt.gca().set_aspect('equal', adjustable='box')
        plt.show()

simulation(Route_etude, Direction_etude, Traffic_etude,temps,100)